In [1]:
#加载所需要的库
!pip install transformers datasets evaluate tokenizer sacrebleu sentencepiece accelerate sacremoses

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 54.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 37.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 59.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.1/219.1 kB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 kB 41.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 82.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB

In [2]:
import numpy as np
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer,AutoConfig
import torch
import evaluate
from datasets import load_dataset
from transformers import AutoTokenizer,DataCollatorForSeq2Seq
#checkpoint = 'Helsinki-NLP//opus-mt-en-zh'
checkpoint = "Jyshen/Translation_en2zh"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [4]:
# Training for DDDSSS dataset
DS_dataset = load_dataset("DDDSSS/en-zh-dataset")
DS_dataset = DS_dataset["train"].train_test_split(test_size=0.2)

def DS_pfunc(examples):
    inputs = [example["en"] for example in examples["translation"]]
    targets = [example["zh"] for example in examples["translation"]]
    model_inputs = tokenizer(inputs, text_target=targets, max_length=512, truncation=True)
    return model_inputs
model_inputs = DS_dataset.map(DS_pfunc, batched=True)

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/DDDSSS___json/DDDSSS--en-zh-dataset-44c5559b2e11d4bb/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

Map:   0%|          | 0/77230 [00:00<?, ? examples/s]

Map:   0%|          | 0/19308 [00:00<?, ? examples/s]

In [6]:
# Training for CoQA dataset
from datasets import concatenate_datasets
from huggingface_hub import notebook_login
notebook_login()

CoQA_dataset = load_dataset("silk-road/Luotuo-QA-A-CoQA-Chinese")
CoQA_dataset = CoQA_dataset["train"].train_test_split(test_size=0.2)

def Co_pfunc(examples):
    inputs = [example for example in examples["story"]]
    targets = [example for example in examples["story_zh"]]
    model_inputs = tokenizer(inputs, text_target=targets, max_length=512, truncation=True)
    return model_inputs
model_inputs2 = CoQA_dataset.map(Co_pfunc, batched=True)
model_inputs['train'] = concatenate_datasets([model_inputs['train'], model_inputs2['train']])
model_inputs['test'] = concatenate_datasets([model_inputs['test'], model_inputs2['test']])

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 3>:3                                                                              │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/datasets/load.py:1773 in load_dataset                    │
│                                                                                                  │
│   1770 │   )                                                                                     │
│   1771 │                                                                                         │
│   1772 │   # Create a dataset builder                                                            │
│ ❱ 1773 │   builder_instance = load_dataset_builder(                                              │
│   1774 │   │   path=path,                                                                        │
│   1775 │   │   name=name,                                                                        │
│   1776 │   │   data_dir=data_dir,                                                                │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/datasets/load.py:1502 in load_dataset_builder            │
│                                                                                                  │
│   1499 │   if use_auth_token is not None:                                                        │
│   1500 │   │   download_config = download_config.copy() if download_config else DownloadConfig(  │
│   1501 │   │   download_config.use_auth_token = use_auth_token                                   │
│ ❱ 1502 │   dataset_module = dataset_module_factory(                                              │
│   1503 │   │   path,                                                                             │
│   1504 │   │   revision=revision,                                                                │
│   1505 │   │   download_config=download_config,                                                  │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/datasets/load.py:1215 in dataset_module_factory          │
│                                                                                                  │
│   1212 │   │   │   │   if isinstance(e1, EmptyDatasetError):                                     │
│   1213 │   │   │   │   │   raise e1 from None                                                    │
│   1214 │   │   │   │   if isinstance(e1, FileNotFoundError):                                     │
│ ❱ 1215 │   │   │   │   │   raise FileNotFoundError(                                              │
│   1216 │   │   │   │   │   │   f"Couldn't find a dataset script at {relative_to_absolute_path(c  │
│   1217 │   │   │   │   │   │   f"Couldn't find '{path}' on the Hugging Face Hub either: {type(e  │
│   1218 │   │   │   │   │   ) from None                                                           │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
FileNotFoundError: Couldn't find a dataset script at 
/content/silk-road/Luotuo-QA-A-CoQA-Chinese/Luotuo-QA-A-CoQA-Chinese.py or any data file in the same directory. 
Couldn't find 'silk-road/Luotuo-QA-A-CoQA-Chinese' on the Hugging Face Hub either: FileNotFoundError: Dataset 
'silk-road/Luotuo-QA-A-CoQA-Chinese' doesn't exist on the Hub. If the repo is private or gated, make sure to log in
with `huggingface-cli login`.

In [ ]:
# Training for Dolly dataset
Dolly_dataset = load_dataset("silk-road/chinese-dolly-15k")
Dolly_dataset = Dolly_dataset["train"].train_test_split(test_size=0.2)

def Do_pfunc(examples):
    inputs = [example for example in examples["instruction"]]
    targets = [example for example in examples["instruction_zh"]]
    model_inputs = tokenizer(inputs, text_target=targets, max_length=512, truncation=True)
    return model_inputs
model_inputs2 = Dolly_dataset.map(Do_pfunc, batched=True)
model_inputs['train'] = concatenate_datasets([model_inputs['train'], model_inputs2['train']])
model_inputs['test'] = concatenate_datasets([model_inputs['test'], model_inputs2['test']])

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-8da581cb8d285022/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

Map:   0%|          | 0/12008 [00:00<?, ? examples/s]

Map:   0%|          | 0/3003 [00:00<?, ? examples/s]

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=checkpoint)
metric = evaluate.load("sacrebleu")

In [ ]:
def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]
    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)
batchsize=8
training_args = Seq2SeqTrainingArguments(
    output_dir="./best_model",
    evaluation_strategy="epoch",
    learning_rate=2e-4,
    per_device_train_batch_size=batchsize,
    per_device_eval_batch_size=batchsize,
    weight_decay=0.01,
    # save_total_limit=3,
    num_train_epochs=2,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=False,
    save_strategy="epoch",
    jit_mode_eval=True
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=model_inputs["train"],
    eval_dataset=model_inputs["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Bleu,Gen Len
1,2.125100,1.433450,12.198000,33.705400
2,1.318000,1.180593,14.918800,34.203500


/usr/local/lib/python3.10/dist-packages/transformers/models/marian/modeling_marian.py:242: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if attn_weights.size() != (bsz * self.num_heads, tgt_len, src_len):
/usr/local/lib/python3.10/dist-packages/transformers/models/marian/modeling_marian.py:249: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if attention_mask.size() != (bsz, 1, tgt_len, src_len):
/usr/local/lib/python3.10/dist-packages/transformers/models/marian/modeling_marian.py:281: TracerWarning: Converting a tensor to a Python boolean might cause th

TrainOutput(global_step=23712, training_loss=1.8893500005024486, metrics={'train_runtime': 8718.6364, 'train_samples_per_second': 21.757, 'train_steps_per_second': 2.72, 'total_flos': 8704482792505344.0, 'train_loss': 1.8893500005024486, 'epoch': 2.0})

In [ ]:
dir = "/content/drive/MyDrive/best_model/"
model.save_pretrained(dir)
tokenizer.save_pretrained(dir)

('/content/drive/MyDrive/best_model/tokenizer_config.json',
 '/content/drive/MyDrive/best_model/special_tokens_map.json',
 '/content/drive/MyDrive/best_model/vocab.json',
 '/content/drive/MyDrive/best_model/added_tokens.json')